In [1]:
from transformer import Transformer  # this is the transformer.py file
import torch
import numpy as np

In [5]:
english_file = "E:\\ML\\transformer\\dataset\\english.txt"  # replace this path with appropriate one
kannada_file = "E:\\ML\\transformer\\dataset\\kannada.txt"  # replace this path with appropriate one

# Generated this by filtering Appendix code

START_TOKEN = "<START>"
PADDING_TOKEN = "<PADDING>"
END_TOKEN = "<END>"

kannada_vocabulary = [
    START_TOKEN,
    " ",
    "!",
    '"',
    "#",
    "$",
    "%",
    "&",
    "'",
    "(",
    ")",
    "*",
    "+",
    ",",
    "-",
    ".",
    "/",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    ":",
    "<",
    "=",
    ">",
    "?",
    "ˌ",
    "ँ",
    "ఆ",
    "ఇ",
    "ా",
    "ి",
    "ీ",
    "ు",
    "ూ",
    "ಅ",
    "ಆ",
    "ಇ",
    "ಈ",
    "ಉ",
    "ಊ",
    "ಋ",
    "ೠ",
    "ಌ",
    "ಎ",
    "ಏ",
    "ಐ",
    "ಒ",
    "ಓ",
    "ಔ",
    "ಕ",
    "ಖ",
    "ಗ",
    "ಘ",
    "ಙ",
    "ಚ",
    "ಛ",
    "ಜ",
    "ಝ",
    "ಞ",
    "ಟ",
    "ಠ",
    "ಡ",
    "ಢ",
    "ಣ",
    "ತ",
    "ಥ",
    "ದ",
    "ಧ",
    "ನ",
    "ಪ",
    "ಫ",
    "ಬ",
    "ಭ",
    "ಮ",
    "ಯ",
    "ರ",
    "ಱ",
    "ಲ",
    "ಳ",
    "ವ",
    "ಶ",
    "ಷ",
    "ಸ",
    "ಹ",
    "಼",
    "ಽ",
    "ಾ",
    "ಿ",
    "ೀ",
    "ು",
    "ೂ",
    "ೃ",
    "ೄ",
    "ೆ",
    "ೇ",
    "ೈ",
    "ೊ",
    "ೋ",
    "ೌ",
    "್",
    "ೕ",
    "ೖ",
    "ೞ",
    "ೣ",
    "ಂ",
    "ಃ",
    "೦",
    "೧",
    "೨",
    "೩",
    "೪",
    "೫",
    "೬",
    "೭",
    "೮",
    "೯",
    PADDING_TOKEN,
    END_TOKEN,
]

english_vocabulary = [
    START_TOKEN,
    " ",
    "!",
    '"',
    "#",
    "$",
    "%",
    "&",
    "'",
    "(",
    ")",
    "*",
    "+",
    ",",
    "-",
    ".",
    "/",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    ":",
    "<",
    "=",
    ">",
    "?",
    "@",
    "[",
    "\\",
    "]",
    "^",
    "_",
    "`",
    "a",
    "b",
    "c",
    "d",
    "e",
    "f",
    "g",
    "h",
    "i",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "p",
    "q",
    "r",
    "s",
    "t",
    "u",
    "v",
    "w",
    "x",
    "y",
    "z",
    "{",
    "|",
    "}",
    "~",
    PADDING_TOKEN,
    END_TOKEN,
]

In [6]:
index_to_kannada = {k: v for k, v in enumerate(kannada_vocabulary)}
kannada_to_index = {v: k for k, v in enumerate(kannada_vocabulary)}
index_to_english = {k: v for k, v in enumerate(english_vocabulary)}
english_to_index = {v: k for k, v in enumerate(english_vocabulary)}

In [7]:
with open(english_file, "r") as file:
    english_sentences = file.readlines()
with open(kannada_file, "r") as file:
    kannada_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip("\n").lower() for sentence in english_sentences]
kannada_sentences = [sentence.rstrip("\n") for sentence in kannada_sentences]

In [8]:
english_sentences[:10]

['hes a scientist.',
 "'but we speak the truth aur ye sach hai ke gujarat mein vikas pagal hogaya hai,'' rahul gandhi further said in banaskantha",
 '8 lakh crore have been looted.',
 'i read a lot into this as well.',
 "she was found dead with the phone's battery exploded close to her head the following morning.",
 'how did mankind come under satans rival sovereignty?',
 'and then i became prime minister.',
 'what about corruption?',
 'no differences',
 '"""the shooting of the film is 90 percent done."']

In [13]:
kannada_sentences[:10]

['ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.',
 '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"',
 'ಕಳ್ಳತನವಾಗಿದ್ದ 8 ಲಕ್ಷ ರೂ.',
 'ಇದರ ಬಗ್ಗೆ ನಾನೂ ಸಾಕಷ್ಟು ಓದಿದ್ದೇನೆ.',
 'ಮಾನವಕುಲವು ಸೈತಾನನ ಆಳಿಕೆಯ ಕೆಳಗೆ ಬಂದದ್ದು ಹೇಗೆ?',
 'ನಂತರ ಪ್ರಧಾನಿ ಕೂಡ ಆಗುತ್ತೇನೆ.',
 'ಭ್ರಷ್ಟಾಚಾರ ಏಕಿದೆ?',
 'ಆ ಚಿತ್ರದ ಶೇ 90ರಷ್ಟು ಚಿತ್ರೀಕರಣವೂ ಈಗಾಗಲೇ ಮುಗಿದು ಹೋಗಿದೆ.',
 'ವಿಶೇಷ ಕಾನೂನು',
 'ಆಗ ಅರಸನು ಗಿತ್ತೀಯನಾದ ಇತ್ತೈಯನ್ನು ನೋಡಿ--ನೀನು ನಮ್ಮ ಸಂಗಡ ಬರುವದು ಯಾಕೆ? ನಿನ್ನ ಸ್ಥಳಕ್ಕೆ ಹಿಂದಿರುಗಿ ಹೋಗಿ ಅರಸನ ಸಂಗಡ ಇರು. ಯಾಕಂದರೆ ನೀನು ಸೆರೆಹಿಡಿಯಲ್ಪಟ್ಟವನಾದ ಅನ್ಯದೇಶದವನು.']

In [14]:
import numpy as np

PERCENTILE = 97
print(
    f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in kannada_sentences], PERCENTILE)}"
)
print(
    f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}"
)

97th percentile length Kannada: 136.0
97th percentile length English: 142.0


In [15]:
max_sequence_length = 200


def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True


def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (
        max_sequence_length - 1
    )  # need to re-add the end token so leaving 1 space


valid_sentence_indicies = []
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = (
        kannada_sentences[index],
        english_sentences[index],
    )
    if (
        is_valid_length(kannada_sentence, max_sequence_length)
        and is_valid_length(english_sentence, max_sequence_length)
        and is_valid_tokens(kannada_sentence, kannada_vocabulary)
    ):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 164022
Number of valid sentences: 164022


In [16]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [10]:
kannada_sentences[:3]

['ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.',
 '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"',
 'ಕಳ್ಳತನವಾಗಿದ್ದ 8 ಲಕ್ಷ ರೂ.']

In [17]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(kannada_vocabulary)

transformer = Transformer(
    d_model,
    ffn_hidden,
    num_heads,
    drop_prob,
    num_layers,
    max_sequence_length,
    kn_vocab_size,
    english_to_index,
    kannada_to_index,
    START_TOKEN,
    END_TOKEN,
    PADDING_TOKEN,
)

In [18]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [19]:
from torch.utils.data import Dataset, DataLoader


class TextDataset(Dataset):
    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [20]:
dataset = TextDataset(english_sentences, kannada_sentences)

In [15]:
len(dataset)

164022

In [21]:
dataset[1]

("'but we speak the truth aur ye sach hai ke gujarat mein vikas pagal hogaya hai,'' rahul gandhi further said in banaskantha",
 '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"')

In [22]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)
len(iterator)

5468

In [23]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('hes a scientist.', "'but we speak the truth aur ye sach hai ke gujarat mein vikas pagal hogaya hai,'' rahul gandhi further said in banaskantha", '8 lakh crore have been looted.', 'i read a lot into this as well.', 'how did mankind come under satans rival sovereignty?', 'and then i became prime minister.', 'what about corruption?', '"""the shooting of the film is 90 percent done."', 'the special statute', '"then the king said to ittai the gittite, ""why do you also go with us? return, and stay with the king. for you are a foreigner, and also an exile. return to your own place."', 'what happened at the un general assembly?', 'the meeting was attended by prime minister narendra modi, home minister amit shah and defence minister rajnath singh, among others.', 'it has been under discussion for a long time.', 'buses cannot get there.', 'why then this tradition was not thought of?', 'kashmiri youth join indian army', 'basic amenities elude this village', 'off-budget borrowings of the state

In [24]:
from torch import nn

criterian = nn.CrossEntropyLoss(
    ignore_index=kannada_to_index[PADDING_TOKEN], reduction="none"
)

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [25]:
NEG_INFTY = -1e9


def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length], True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full(
        [num_sentences, max_sequence_length, max_sequence_length], False
    )
    decoder_padding_mask_self_attention = torch.full(
        [num_sentences, max_sequence_length, max_sequence_length], False
    )
    decoder_padding_mask_cross_attention = torch.full(
        [num_sentences, max_sequence_length, max_sequence_length], False
    )

    for idx in range(num_sentences):
        eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(
            kn_batch[idx]
        )
        eng_chars_to_padding_mask = np.arange(
            eng_sentence_length + 1, max_sequence_length
        )
        kn_chars_to_padding_mask = np.arange(
            kn_sentence_length + 1, max_sequence_length
        )
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(
        look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0
    )
    decoder_cross_attention_mask = torch.where(
        decoder_padding_mask_cross_attention, NEG_INFTY, 0
    )
    return (
        encoder_self_attention_mask,
        decoder_self_attention_mask,
        decoder_cross_attention_mask,
    )

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.
 

Note the target mask starts with 2 rows of non masked items: https://github.com/SamLynnEvans/Transformer/blob/master/Beam.py#L55


In [26]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        (
            encoder_self_attention_mask,
            decoder_self_attention_mask,
            decoder_cross_attention_mask,
        ) = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(
            eng_batch,
            kn_batch,
            encoder_self_attention_mask.to(device),
            decoder_self_attention_mask.to(device),
            decoder_cross_attention_mask.to(device),
            enc_start_token=False,
            enc_end_token=False,
            dec_start_token=True,
            dec_end_token=True,
        )
        labels = transformer.decoder.sentence_embedding.batch_tokenize(
            kn_batch, start_token=False, end_token=True
        )
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device),
        ).to(device)
        valid_indicies = torch.where(
            labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True
        )
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        # train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Kannada Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
                if idx == kannada_to_index[END_TOKEN]:
                    break
                predicted_sentence += index_to_kannada[idx.item()]
            print(f"Kannada Prediction: {predicted_sentence}")

            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                (
                    encoder_self_attention_mask,
                    decoder_self_attention_mask,
                    decoder_cross_attention_mask,
                ) = create_masks(eng_sentence, kn_sentence)
                predictions = transformer(
                    eng_sentence,
                    kn_sentence,
                    encoder_self_attention_mask.to(device),
                    decoder_self_attention_mask.to(device),
                    decoder_cross_attention_mask.to(device),
                    enc_start_token=False,
                    enc_end_token=False,
                    dec_start_token=True,
                    dec_end_token=False,
                )
                next_token_prob_distribution = predictions[0][
                    word_counter
                ]  # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token,)
                if next_token == END_TOKEN:
                    break

            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.489313125610352
English: hes a scientist.
Kannada Translation: ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.
Kannada Prediction: ೃೃँँೃರೃೃ,+,,3ి>ೃಥಙೃೃు3+ೃँి,,+,+ಥಏಥ೧3೧ೄँ(ँ,3ಥ,್3್ँ್್್ಏ್್ಥ,,(್ಥಥ೧೧ँ,ಥँ.ˌ,,,೧ಥ,ಥೄ,,೧ಥ,,,,,ಥಥಥಥಥಥಥಥ,ೄೄೄೄೄ,ೄ,ೄೄ೧ೄಱ,,ೄ>>೧ँೄಶಶಶ೧ಥಥ೧ೄ೧ಥಥಥೄಥಥಥ,ಥಥಥಥಥಥಥಥಥ,,,ಥ,ಳ್ಇುಙಇಥಥಥ,ಥ,ಇ,್೧್೧ಥ<,<ಇ<,,,,<<<<<೧<<<ಙ್<<<<<<೭೭<<<್್
Evaluation translation (should we go to the mall?) : ('                  ್್್           ್್್      ್್್್್್  ್್್್್್ ್್್್್              ್್್್್್್್್ ್್್್್್್್್್್್್್್್್್ ್್್್್್್       ರರ್್್್್್್  ್್್್            ್್್್್   ್್್್್್್್್್್           ್್್್್್್್್್್್್್್್್್್್್್್್್',)
-------------------------------------------
Iteration 100 : 3.5280466079711914
English: she ate it.
Kannada Translation: ಅವಳು ಅವನಿಗೆ ಊಟ ಹಾಕಿದಳೂ.
Kannada Prediction: ನರದ್   ್ ್ ್್  ಿ್  ಿ್
Evaluation translation (should we go to the mall?) : ('ಕಕ್                   ು  ಿ ಿ <END>',)
-------------------------------------------
Iteration 200 : 3.2629799842834473
English: caste and religion were unknown.
Kannada 

KeyboardInterrupt: 

## Inference

In [ ]:
transformer.eval()


def translate(eng_sentence):
    eng_sentence = (eng_sentence,)
    kn_sentence = ("",)
    for word_counter in range(max_sequence_length):
        (
            encoder_self_attention_mask,
            decoder_self_attention_mask,
            decoder_cross_attention_mask,
        ) = create_masks(eng_sentence, kn_sentence)
        predictions = transformer(
            eng_sentence,
            kn_sentence,
            encoder_self_attention_mask.to(device),
            decoder_self_attention_mask.to(device),
            decoder_cross_attention_mask.to(device),
            enc_start_token=False,
            enc_end_token=False,
            dec_start_token=True,
            dec_end_token=False,
        )
        next_token_prob_distribution = predictions[0][word_counter]
        next_token_index = torch.argmax(next_token_prob_distribution).item()
        next_token = index_to_kannada[next_token_index]
        kn_sentence = (kn_sentence[0] + next_token,)
        if next_token == END_TOKEN:
            break
    return kn_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)
# ದಿನ ಪ್ರಾರಂಭವಾದಾಗ ನಾವು ಏನು ಮಾಡಬೇಕು?

In [ ]:
translation = translate("how is this the truth?")
print(translation)
# ಇದು ಹೇಗೆ ಸತ್ಯ

In [ ]:
translation = translate("the world is a large place with different people")
print(translation)
# ಪ್ರಪಂಚವು ವಿಭಿನ್ನ ಜನರೊಂದಿಗೆ ದೊಡ್ಡ ಸ್ಥಳವಾಗಿದೆ

In [ ]:
translation = translate("my name is ajay")
print(translation)
# ನನ್ನ ಹೆಸರು ಅಜಯ್

In [ ]:
translation = translate("i cannot stand this smell")
print(translation)
# ನಾನು ಈ ವಾಸನೆಯನ್ನು ಸಹಿಸುವುದಿಲ್ಲ

In [ ]:
translation = translate("noodles are the best")
print(translation)

In [ ]:
translation = translate("why care about this?")
print(translation)

This translated pretty well : "What is the reason. Why" without punctuation.

In [ ]:
translation = translate("this is the best thing ever")
print(translation)
# ಇದು ಎಂದೆಂದಿಗೂ ಉತ್ತಮವಾಗಿದೆ

The translation : "This is very unusual"

In [ ]:
translation = translate("i am here")
print(translation)
# ನಾನು ಇಲ್ಲಿದ್ದೇನೆ

Translation: "I have heard". 
This is why word based translator may perform better than character translator. This is actually very good at optimizing the objective of the current transformer even though the translation is off.

In [ ]:
translation = translate("click this")
print(translation)
# ಇದನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ

In [ ]:
translation = translate("where is the mall?")
print(translation)

In [ ]:
translation = translate("what should we do?")
print(translation)

This is correct; but it absolutely fumbles on the next one

In [ ]:
translation = translate("today, what should we do")
print(translation)

In [ ]:
translation = translate("why did they activate?")
print(translation)
# ಅವರು ಏಕೆ ಸಕ್ರಿಯಗೊಳಿಸಿದರು?

In [ ]:
translation = translate("why did they do this?")
print(translation)
# ಅವರು ಇದನ್ನು ಏಕೆ ಮಾಡಿದರು?

That turned out well!

In [ ]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

Translation: "I will give you something"

In [ ]:
translation = translate("whats the word on the street?")
print(translation)

Kind of close semantically. Translation is something like: "What is this about"

## Insights

- When training, we can treat every alphabet as a single unit instead of splitting it into it's corresponding parts to preserve meaning. For example, ಮಾ should be 1 unit when comuting a loss. It should not be decomposed into ಮ + ఆ
- Using word-based or BPE based tokenizations may help mitigate (1). Also, we will get valid word (or BPE) units if we do so. 
- Make sure the training set has a large variety of sentences that are not just about one topic like "work" and "government"
- Increase the number of encoder / decoder units for better translations. It was set to the minimum of 1 of each unit here.
- Create a translator with a language you understand ideally.

Overall, this model definately learned something. And you can use other languages instead of this kannada language and might see better luck